In [1]:
from openstack import connection, identity
import copy
import re
import openpyxl
from openpyxl import *
import os
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import colors, Alignment
from openpyxl.cell import Cell

In [2]:
def connection_to_cluster(project):
    conn = connection.Connection(auth_url="http://controller:35357/v3",
                                 project_name=project,
                                 project_domain_name="default",
                                 user_domain_name="default",
                                 username="admin",
                                 password="QIP@t4Ay")
    return conn

In [3]:
class OSmap(object):
    
    def __init__ (self, connection='', OS=True, with_flavors=True, with_volumes=True, hypervisor_details=False):
        'Инит функция'
        if OS == True:
            self.__conn = connection
            self.__sl = [i for i in self.__conn.compute.servers(all_tenants=True)]
            self.__hl = [i for i in self.__conn.compute.hypervisors()]
            
            if with_flavors == True:
                self.__fd = self.__get_fl_dict()
                self.dict_flavors = copy.deepcopy(self.__fd)
            
            if with_volumes == True:
                self.__vd = self.__get_vl_dict()
                self.dict_volumes = copy.deepcopy(self.__vd)
                
                
            self.dict_hypervisors = self.__get_hp_dict(details=hypervisor_details)
            self.dict_servers = self.__get_vm_dict(flavors=with_flavors)
            self.list_names  = self.__get_names()
    
    
    def get_vm_by_name(self, vmn=''):
        'Функция возвращает объект ВМ на основании имени'
        for i in self.__sl:
            if i.name == vmn:
                return i 
    
    def get_vm_by_id(self, vmn=''):
        'Функция возвращает объект ВМ на основании ID'
        for i in self.__sl:
            if i.id == vmn:
                return i 
    
        
    
    def __get_hp_dict(self, details=False):
        'Функция возвращает словарь гипервизоров'
        d = {}
        for i in self.__hl:
            if details == True:
                d[i.name]=self.__conn.compute.get_hypervisor(i.id).to_dict()
            else:
                d[i.name]=i.to_dict()
        return d   
           
    def __get_fl_dict(self):
        'Функция возвращает словарь флейворов'
        d = {}
        for i in set([i.flavor['id'] for i in self.__sl]):
            d[i]=self.__conn.compute.get_flavor(flavor=i).to_dict()
        return d
    
    def __get_vl_dict(self):
        'Функция возвращает словарь вольюмов'
        d = {}
        for i in [i for i in self.__conn.block_store.volumes(all_tenants=True)]:
            d[i.id]=i.to_dict()
        return d
    
    
    def __get_names (self):
        'Функция возвращает список имен ВМ'
        a = [i.name for i in self.__sl]
        return a
    
            
    def __get_vm_dict(self,flavors=True):
        'Возвращает словарь с описанием ВМ из OS + cpu & ram'
        d = {}
        for i in self.__sl:
            d[i.name] = self.get_vm_by_name(vmn=i.name).to_dict()
            if flavors == True:
                d[i.name]['cpu'] = self.__fd[i.flavor['id']]['vcpus']
                d[i.name]['ram'] = self.__fd[i.flavor['id']]['ram']
        return d
        

In [4]:
conn = connection_to_cluster('admin')

In [5]:
#%%time
#x = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False)

In [6]:
#%%time
#y = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False)

In [7]:
#%%time
#z = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True)

In [8]:
#%%time
#x1 = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False, hypervisor_details=True)

In [9]:
#%%time
#y1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False, hypervisor_details=True)

In [10]:
%%time
z1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True, hypervisor_details=True)

CPU times: user 4.81 s, sys: 214 ms, total: 5.02 s
Wall time: 40.5 s


## Копируем данные из класса. Создаем словарь групп

In [11]:
## Типы распределений
## 1 - распределение по зонам доступности
## 2 - распределение по ВУ
## 3 - распределение по модулю 2 (четные на одной ЗД, нечетные на другой ЗД)
## 4 - распределение по модулю 3 (1,2,3 на разных ЗД)
## 5 - распределение - master-stanby на разных ЗД

In [12]:
dRealVm = copy.deepcopy(z1.dict_servers)

In [13]:
lRealVmName = copy.deepcopy(z1.list_names)

In [14]:
dRealHyper = copy.deepcopy(z1.dict_hypervisors)

In [15]:
master_slave_list = [\
                    ['pg-master-standby','pg-master'],\
                    ['pg-attribute-standby-books','pg-attribute-books'],\
                    ['pg-attribute-standby-borders','pg-attribute-borders'],\
                    ['pg-attribute-standby-right-registry-entities','pg-attribute-right-registry-entities'],\
                    ['pg-reservation-01-standby','pg-reservation-01'],\
                     ['pg-master-replica-standy-01','pg-master-replica-01']\
                    
                    ]

In [16]:
master_slave_list
total_ms_list = [i for j in master_slave_list for i in j]

In [17]:
[i for i in master_slave_list if 'pg-master-standby' in i ][0]

['pg-master-standby', 'pg-master']

In [18]:

with open('./group_prefix', 'r') as f:
    lGroupPrefix = f.readlines()
    lGroupPrefix = [i.replace('\n','') for i in lGroupPrefix]
f.close()


with open('./group_prefixes_via_types2', 'r') as f:
    lGroupPrefixTypes = f.readlines()
    lGroupPrefixTypes = [i.replace('\n','') for i in lGroupPrefixTypes]
    lGroupPrefixTypes = [i.split(',') for i in lGroupPrefixTypes]
f.close()


with open('./z1', 'r') as f:
    lAZ1 = f.readlines()
    lAZ1 = [i.replace('\n','') for i in lAZ1]
f.close()


with open('./z2', 'r') as f:
    lAZ2 = f.readlines()
    lAZ2 = [i.replace('\n','') for i in lAZ2]
f.close()
    
with open('./z3', 'r') as f:
    lAZ3 = f.readlines()
    lAZ3 = [i.replace('\n','') for i in lAZ3]
f.close()

with open('./very_new_cn_all', 'r') as f:
    lCN = f.readlines()
    lCN = [i.replace('\n','') for i in lCN]
    lCN = [i.split(',') for i in lCN]
f.close()

with open('./very_new_cn_migr', 'r') as f:
    lMigr = f.readlines()
    lMigr = [i.replace('\n','') for i in lMigr]
    lMigr = [i.split(',') for i in lMigr]
    lMigr = [i[1] for i in lMigr]
f.close()

In [19]:

dGroups = {}
for lprefix in lGroupPrefixTypes:
    prefix_name=lprefix[0]
    prefix_type=int(lprefix[1])
    sTemp = re.escape(prefix_name) + r"-\d{2,}"
    #print(prefix)
    regexp = re.compile(sTemp, re.IGNORECASE)
    dGroups[prefix_name]={}
    dGroups[prefix_name]['type'] = prefix_type
    dGroups[prefix_name]['vms'] = []
    for vm in lRealVmName:
        if regexp.match(vm):
            #print(vm)
            dGroups[prefix_name]['vms'].append(vm)
    dGroups[prefix_name]['vm_count'] = len(dGroups[prefix_name]['vms'])

In [20]:
#[[i, dGroups[i]['vm_count']] for i in dGroups.keys() if dGroups[i]['type'] == 1]


In [21]:
def make_xls():
    file="my_big_table.xlsx"
    wb = Workbook ()
    try:
        os.remove(file)
    except OSError:
        pass
    ws = wb.active
    ws.title=('Распределение ВМ по ВУ')
    warningFill = PatternFill(start_color='ffcc33',
                       end_color='ffcc33', fill_type='solid')
    criticalFill = PatternFill(start_color='ee4000',
                       end_color='ee4400', fill_type='solid')
    headersFill = PatternFill(start_color='ebecec',
                       end_color='ebecec', fill_type='solid')
    vm_headersFill = PatternFill(start_color='faebd7',
                       end_color='faebd7', fill_type='solid')
    top_headersFill = PatternFill(start_color='8c98a0',
                       end_color='8c98a0', fill_type='solid')

    row = 3
    col = 1
    t=dUseHyper
    ws.merge_cells('A1:E1')

    for i in t.keys():
        for rows in ws.iter_rows(min_row=row - 1, max_row=row - 1 , min_col=1, max_col=7):
            for cell in rows:
                cell.font = Font(italic=True)
                cell.alignment= Alignment(horizontal='center')
                cell.fill = top_headersFill
        ws.cell(column = col, row = row - 1 , value='Имя ВУ').alignment= Alignment(horizontal='left')
        ws.cell(column = col + 1, row = row  - 1, value='Номер зоны доступности')
        ws.cell(column = col + 2, row = row - 1, value='Переподписка по CPU')
        ws.cell(column = col + 3, row = row - 1, value='Свободная RAM')
        ws.cell(column = col + 4, row = row - 1, value='Количество ВМ на ВУ')
        for rows in ws.iter_rows(min_row=row + 1, max_row=row + 1 , min_col=1, max_col=7):
            for cell in rows:
                cell.font = Font(italic=True) 
                cell.alignment= Alignment(horizontal='right')
        ws.cell(column=col, row=row).font = Font(name='Calibri', bold=True)
        for rows in ws.iter_rows(min_row=row , max_row=row, min_col=1, max_col=7):
            for cell in rows:
                  cell.fill = headersFill 
        ws.cell(column=col, row=row, value=t[i]['label'])
        ws.cell(column=col + 1, row=row, value=t[i]['az'])
        ws.cell(column=col + 2, row=row, value="{:10.2f}".format((t[i]['free_cpu']))).alignment = Alignment(horizontal='center')
        ws.cell(column=col + 3, row=row, value="{:10.2f}".format((t[i]['free_ram']))).alignment = Alignment(horizontal='center')
        ws.cell(column=col + 4, row=row, value=t[i]['vm_count'])
       # ws.cell(column=col + 5, row=row, value=t[i]['vm_critical']) 
       # ws.cell(column=col + 6, row=row, value=t[i]['vm_warning']) 
        row = row + 1
        for rows in ws.iter_rows(min_row=row, max_row=row, min_col=1, max_col=7):
            for cell in rows:
                cell.fill = vm_headersFill
                cell.alignment = Alignment(horizontal='center')
        ws.cell(column=col + 2, row=row, value='Имя ВМ').alignment = Alignment(horizontal='left')
        ws.cell(column=col + 3, row=row, value='vCPU')
        ws.cell(column=col + 4, row=row, value='RAM') 
        ws.cell(column=col + 5, row=row, value='Имя логической группы') 

        row = row + 1
        for j in t[i]['vms']:
            v = dUseVms[j]
            ws.cell(column=col + 2, row=row, value=v['name'])
            ws.cell(column=col + 3, row=row, value=v['cpu'])
            ws.cell(column=col + 4, row=row, value=v['ram']) 
            #if v['crit_rate'] == 2:
            #    crit = 1
            #elif v['crit_rate'] == 1:
            #    crit = 2
            #elif v['crit_rate'] == 0:
            #    crit = 'н/д'
            #ws.cell(column=col + 5, row=row, value=crit)
            #if v['crit_rate'] == 2:
            #    for rows in ws.iter_rows(min_row=row, max_row=row, min_col=3, max_col=7):
            #        for cell in rows:
            #              cell.fill = criticalFill
            #elif v['crit_rate'] == 1:
            #    for rows in ws.iter_rows(min_row=row, max_row=row, min_col=3, max_col=7):
            #        for cell in rows:
            #              cell.fill = warningFill

            ws.cell(column=col + 5, row=row, value=v['group']) 
            row = row + 1
        row = row + 2
    wb.save(file)
    wb.close() 


In [22]:
### основные функции по созданию словарей

def create_dict_computes_clean(z1,z2,z3,cn,migr):
    rd={}
    for i in cn:
        hname = i[0]
        hlabel = i[1]
        #print(hlabel,hname)
        if hlabel in z1:
            hzone=int('1')
        elif hlabel in z2:
            hzone=int('2')
        elif hlabel in z3:
            hzone=int('3')
       
        if hlabel in migr:
            hmigr=True
        else: 
            hmigr=False
        
        rd[hname]={}
        rd[hname]['migr']=hmigr
        rd[hname]['label']=hlabel
        rd[hname]['az']=hzone
        rd[hname]['total_cpu']=dRealHyper[hname]['vcpus']
        rd[hname]['total_ram']=dRealHyper[hname]['memory_size']
        rd[hname]['free_cpu']=dRealHyper[hname]['vcpus']
        rd[hname]['free_ram']=dRealHyper[hname]['memory_size']
        rd[hname]['vms']=[]
        rd[hname]['vm_count']=len(rd[hname]['vms'])
        
    return rd
        
def create_dict_vm_with_groups(vdict,gdict):
    rd={}
    for v in vdict.keys():
        rd[v]={}
        rd[v]['cpu']=vdict[v]['cpu']
        rd[v]['name']=vdict[v]['name']
        rd[v]['ram']=vdict[v]['ram']
        rd[v]['cn']=vdict[v]['hypervisor_hostname']
        rd[v]['cn_new'] = ''
        rd[v]['group']=''
        if v in total_ms_list:
            rd[v]['group'] = 'master_slave'
        for g in gdict.keys():
            if v in gdict[g]['vms']:
                rd[v]['group'] = g
                continue
            
    return rd

def create_dict_groups_via_az(gdict):
    rd={}
    rd=copy.deepcopy(gdict)
    for g in gdict:
        rd[g]['az']={}
        rd[g]['az'][1] = int('0')
        rd[g]['az'][2] = int('0')
        rd[g]['az'][3] = int('0')
        rd[g]['az_even'] = ''
        rd[g]['az_odd'] = ''
        rd[g]['hypers']={}
        for h in dVirtualHyper.keys():
            rd[g]['hypers'][h]=0

    
    return rd

In [23]:
##
## Функции распределения и поиска пула доступных гипервизоров
##



###### Необходимо сделать - описать дистрибуцию уровня 2 и 1, дистрибуцию для остальных машин (тип 0). 


def distributuion_init(gdict,vdict,hdict,vms_on_migr):
    global dUseGroups, dUseVms, dUseHyper
    dUseGroups = copy.deepcopy(gdict)
    dUseVms    = copy.deepcopy(vdict)
    dUseHyper  = copy.deepcopy(hdict)
    dl=[]
    for v in dVirtualVm.keys():
        vmname = v
        if   dVirtualVm[v]['group'] == 'master_slave':
            vmtype = int('5')
        elif dVirtualVm[v]['group'] != '':
            vmtype = dVirtualGroups[dVirtualVm[v]['group']]['type']
        else:
            vmtype= int('0')
        dl.append([vmname,vmtype])
    
    distribution(vl=sorted(dl, key = lambda x: float(x[1]), reverse=True))
    
    
def distribution(vl):
    for v in vl:
        vmname = v[0]
        vmtype = v[1]
        hyper_list=[]
        if vmname in vms_on_migr:
            #print( dUseVms[vmname]['cn'])
            hname = dUseVms[vmname]['cn']
            count_after_distrib(h=hname,vm=vmname)
            continue
        if vmtype == 1:
            hyper_list = az_distribution(vmname)
        
        elif vmtype == 2:
            hyper_list = cn_distribution(vmname)
        
        elif vmtype == 3:
            hyper_list = even_distribution(vmname)
            
        elif vmtype == 4: 
            hyper_list = triple_distribution(vmname)
            
        elif vmtype == 5:
            hyper_list = master_slave_distribution(vmname)
            
        elif vmtype == 0:
            hyper_list = null_type_distribution(vmname)
        #print(vmname,vmtype)
        #print(vmname, set([dUseHyper[i]['az'] for i in hyper_list]))
        print(vmname,vmtype)
        main_distributuion_function(hl=hyper_list,vm=vmname)
        
        
def count_after_distrib(h,vm):
    az = dUseHyper[h]['az']
    vm_group = dUseVms[vm]['group']
    if vm_group != '' and vm_group != 'master_slave':
        dUseGroups[vm_group]['az'][az] = dUseGroups[vm_group]['az'][az] + 1
        dUseGroups[vm_group]['hypers'][h] = dUseGroups[vm_group]['hypers'][h] + 1
    vm_cpu = dUseVms[vm]['cpu']
    vm_ram = dUseVms[vm]['ram']
    dUseHyper[h]['free_ram'] = dUseHyper[h]['free_ram'] - vm_ram
    dUseHyper[h]['free_cpu'] = dUseHyper[h]['free_cpu'] - vm_cpu
    dUseHyper[h]['vms'].append(vm)
    dUseHyper[h]['vm_count']=len(dUseHyper[h]['vms'])
    dUseVms[vm]['cn_new']=h
    pass


def main_distributuion_function(hl, vm):
    #print(vm)
    top_list=[]
    for h in hl:
        if dUseHyper[h]['migr']:
            #print('ouch! migration')
            continue
        pram = dUseHyper[h]['free_ram']/dUseHyper[h]['total_ram']
        pcpu = dUseHyper[h]['free_cpu']/dUseHyper[h]['total_cpu']
        res_score = (0.00001 * pcpu + 1 * pram)
        top_list.append([res_score,h])
    #if vm == 'pg-attribute-18':
    #    print (top_list, hl)
    #print(top_list[:1])
    best_buddy = sorted(top_list, reverse=True)[:1][0][1]
   
    count_after_distrib(h=best_buddy,vm=vm)
    
    




In [24]:
#def keep_vm_migrate_on_its_host(vm):
#    rl = list(dUseVms[vm]['cn'])
#    return rl
#    pass

def master_slave_distribution(vm):
    pair = [i for i in master_slave_list if vm in i ][0]
    #pair = [i for i in master_slave_list if vm in i ][0]
    pair_vm = [i for i in pair if i != vm][0]
    pair_cn = dUseVms[pair_vm]['cn_new']
    if pair_cn != '':
        rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] != dUseHyper[pair_cn]['az']]
    else:
        rl = [i for i in dUseHyper.keys()]
    return rl
    pass

def null_type_distribution(vm):
    rl = [i for i in dUseHyper.keys() if not dUseHyper[i]['migr']]
    return rl
    pass

def az_distribution(vm):
    vm_group = dUseVms[vm]['group']
    azl = [dUseGroups[vm_group]['az'][i] for i in dUseGroups[vm_group]['az'].keys()]
    min_az = min(azl)
    min_az_list = [i for i in dUseGroups[vm_group]['az'].keys() if dUseGroups[vm_group]['az'][i] == min_az ]
    rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] in min_az_list]
    #print(vm,min_az,min_az_list)
    return rl
    
    pass

def cn_distribution(vm):
    vm_group = dUseVms[vm]['group']
    cnl = [dUseGroups[vm_group]['hypers'][i] for i in dUseGroups[vm_group]['hypers'].keys()\
          if not dUseHyper[i]['migr']]
    min_cnl = min(cnl)
    rl = [i for i in dUseGroups[vm_group]['hypers'].keys() if dUseGroups[vm_group]['hypers'][i] == min_cnl ]
    #print(min_cnl,rl)
    return rl
    pass



def even_distribution(vm):
    
    group = dUseVms[vm]['group']
    #ga1 = dUseGroups[group]['az1']
    #ga2 = dUseGroups[group]['az2']
    #ga3 = dUseGroups[group]['az3']
    if dUseGroups[group]['az_even'] == '':
        dUseGroups[group]['az_even'] = get_best_availability_zone(group=group)
    if dUseGroups[group]['az_odd'] == '':
        dUseGroups[group]['az_odd'] = get_best_availability_zone(group=group)
    odd_zone = dUseGroups[group]['az_odd']
    even_zone = dUseGroups[group]['az_even']
    stripped_name = vm.strip(group+'-')
    #print(group, )
    regexp = re.compile(r"\d\d-\d\d")
    if regexp.match(stripped_name):
        vmnumber_in_group = int(stripped_name.split('-')[1])
    else:
        vmnumber_in_group = int(stripped_name)
    #print(vm, vmnumber_in_group)
    if vmnumber_in_group % 2 == 0:
        az_to_land = even_zone
    else:
        az_to_land = odd_zone
    rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] == az_to_land]
    return rl





def triple_distribution(vm):
    #print(vm, 'TRIPLE')
    group = dUseVms[vm]['group']
    #ga1 = dUseGroups[group]['az1']
    #ga2 = dUseGroups[group]['az2']
    #ga3 = dUseGroups[group]['az3']
    vmnumber_in_group = int(vm.strip(group+'-'))
    az_to_land = (vmnumber_in_group % 3) + 1
    rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] == az_to_land]
    return rl



def get_best_availability_zone(group):
    azl = [i for i in [1,2,3] if i not in [dUseGroups[group]['az_odd'], dUseGroups[group]['az_even']]]
    alist=[]
    for a in azl:
        memory=sum([dUseHyper[i]['free_ram'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == a and\
                    not dUseHyper[i]['migr']])
        alist.append([memory,a])
    #print(sorted(alist, reverse=True))
    az = sorted(alist, reverse=True)[:1][0][1]
    print(az)
    return az

In [25]:
def count_all(dh,dv,dg):
    l = [dh[h]['free_ram']/dh[h]['total_ram']  for h in dh.keys()]
    m = [dh[h]['free_cpu']/dh[h]['total_cpu']  for h in dh.keys()]
    #m = [hpd_res[h]['perc_cpu'] for h in hpd_res.keys()]
    vml = sum([dh[h]['vm_count'] for h in dh.keys()])
    hpl = [i for i in dh.keys()]
    #print(hpl)
    print("Всего машин:", vml)
    #print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
    #print("На одном ВУ CPU :", cpu_per_cn, ", RAM :", ram_per_cn, "MB")
    print("Всего ВУ:", len(hpl))
    #print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
    #print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

    print("Забитость ВУ по RAM")
    print("Самый забитый :", 1-min(l))
    print("Наименее забитый :", 1-max(l))
    print("Средняя загруженность :",  1 - sum(l)/len(l))
    print('')
    print("Перерасход vCPU относительно CPU ВУ ")
    print("Самый забитый :", 1-min(m))
    print("Наименее забитый :", 1-max(m))
    print("Средняя загруженность :", 1 - sum(m)/len(m))

In [26]:
#dUseHyper

In [27]:
#dRealHyper['cn-d720.egron.tnx']['memory_size']

In [28]:
###
### создаем виртуальную среду на основе базовых словарей
###

dVirtualHyper = create_dict_computes_clean(z1=lAZ1,z2=lAZ2,z3=lAZ3,migr=lMigr,cn=lCN)
dVirtualVm = create_dict_vm_with_groups(vdict=dRealVm,gdict=dGroups)
dVirtualGroups = create_dict_groups_via_az(gdict=dGroups)
ml = [i for i in dVirtualHyper.keys() if dVirtualHyper[i]['migr'] == True]
vms_on_migr = [i for j in ml for i in dVirtualVm.keys() if dVirtualVm[i]['cn'] == j]

In [29]:
#len([i for i in dVirtualVm.keys() if dVirtualVm[i]['group'] == 'master_slave'])

In [30]:
distributuion_init(gdict=dVirtualGroups,vdict=dVirtualVm,hdict=dVirtualHyper,vms_on_migr=vms_on_migr)

pg-master-replica-standy-01 5
pg-attribute-right-registry-entities 5
pg-attribute-standby-right-registry-entities 5
pg-reservation-01-standby 5
pg-attribute-standby-borders 5
pg-attribute-standby-books 5
pg-attribute-borders 5
pg-attribute-books 5
pg-master 5
pg-master-replica-01 5
pg-reservation-01 5
pg-master-standby 5
pkurp-app-039 4
pkurp-app-038 4
pkurp-app-037 4
pkurp-app-036 4
pkurp-app-035 4
pkurp-app-034 4
pkurp-app-033 4
pkurp-app-032 4
pkurp-app-031 4
pkurp-app-030 4
pkurp-app-029 4
pkurp-app-028 4
pkurp-app-027 4
pkurp-app-026 4
pkurp-app-025 4
pkurp-app-024 4
pkurp-app-023 4
pkurp-app-022 4
pkurp-app-021 4
ppoz-db-fs-bkp-10 4
ppoz-db-fs-bkp-09 4
ppoz-db-fs-bkp-08 4
ppoz-db-fs-bkp-07 4
ppoz-db-fs-bkp-06 4
ppoz-db-fs-bkp-05 4
ppoz-db-fs-bkp-04 4
ppoz-db-fs-bkp-03 4
ppoz-db-fs-bkp-02 4
ppoz-db-fs-bkp-01 4
ppoz-db-fs-30 4
ppoz-db-fs-29 4
ppoz-db-fs-28 4
ppoz-db-fs-27 4
ppoz-db-fs-26 4
ppoz-db-fs-25 4
ppoz-db-fs-24 4
ppoz-db-fs-23 4
ppoz-db-fs-22 4
ppoz-db-fs-21 4
ppoz-db-fs-20

isurz-writer-076 1
isurz-writer-071 1
isurz-writer-069 1
isurz-writer-067 1
isurz-writer-064 1
isurz-writer-061 1
isurz-writer-060 1
isurz-writer-057 1
isurz-writer-048 1
isurz-writer-046 1
isurz-writer-040 1
isurz-writer-037 1
isurz-writer-036 1
isurz-writer-034 1
isurz-writer-033 1
isurz-writer-032 1
isurz-writer-031 1
isurz-writer-030 1
isurz-writer-008 1
isurz-writer-001 1
isurz-writer-000 1
isurz-uploader-balancer-01 1
isurz-api-balancer-01 1
isurz-api-06 1
isurz-api-05 1
isurz-api-04 1
isurz-api-03 1
isurz-api-02 1
Isurz-api-01 1
pkurp-redis-001 1
ppoz-filestorage-bal-02 1
ppoz-filestorage-bal-01 1
ppoz-filestorage-04 1
ppoz-filestorage-03 1
ppoz-filestorage-02 1
ppoz-filestorage-01 1
isurz-uploader-076 1
isurz-uploader-071 1
isurz-uploader-069 1
isurz-uploader-067 1
isurz-uploader-064 1
isurz-uploader-061 1
isurz-uploader-060 1
isurz-uploader-057 1
isurz-uploader-048 1
isurz-uploader-046 1
isurz-uploader-040 1
isurz-uploader-037 1
isurz-uploader-036 1
isurz-uploader-034 1
isurz-

In [31]:
count_all(dg=dUseGroups,dh=dUseHyper,dv=dUseVms)

Всего машин: 1661
Всего ВУ: 76
Забитость ВУ по RAM
Самый забитый : 0.982425191976634
Наименее забитый : 0.8239695158513705
Средняя загруженность : 0.8577022473136381

Перерасход vCPU относительно CPU ВУ 
Самый забитый : 3.357142857142857
Наименее забитый : 2.142857142857143
Средняя загруженность : 2.792763157894737


In [32]:
dUseVms['pg-attribute-standby-02']

{'cn': 'cn-db0a.egron.tnx',
 'cn_new': 'cn-db3e.egron.tnx',
 'cpu': 8,
 'group': 'pg-attribute-standby',
 'name': 'pg-attribute-standby-02',
 'ram': 16384}

In [33]:
#rm = [1,2,3,4,511,128,-1,-1,-1]
#min(rm)

In [34]:
make_xls()

In [35]:
#dUseVms
len(master_slave_list)

6

In [36]:
for c in range(0,len(master_slave_list)):
    print(c)
    pair = [i for i in master_slave_list][c]
    print(pair)
    vm1 = pair[0]
    vm2 = pair[1]
    cn1 = dUseVms[vm1]['cn_new']
    cn2 = dUseVms[vm2]['cn_new']
    az1 = dUseHyper[cn1]['az'] 
    az2 = dUseHyper[cn2]['az']

    print(vm1,vm2)
    print(cn1,cn2)
    print(az1,az2)

0
['pg-master-standby', 'pg-master']
pg-master-standby pg-master
cn-e82a.egron.tnx cn-ed8c.egron.tnx
3 2
1
['pg-attribute-standby-books', 'pg-attribute-books']
pg-attribute-standby-books pg-attribute-books
cn-ed8e.egron.tnx cn-e8aa.egron.tnx
2 3
2
['pg-attribute-standby-borders', 'pg-attribute-borders']
pg-attribute-standby-borders pg-attribute-borders
cn-ee38.egron.tnx cn-e8ae.egron.tnx
2 3
3
['pg-attribute-standby-right-registry-entities', 'pg-attribute-right-registry-entities']
pg-attribute-standby-right-registry-entities pg-attribute-right-registry-entities
cn-e8b6.egron.tnx cn-fdec.egron.tnx
3 2
4
['pg-reservation-01-standby', 'pg-reservation-01']
pg-reservation-01-standby pg-reservation-01
cn-fdc6.egron.tnx cn-e89a.egron.tnx
2 3
5
['pg-master-replica-standy-01', 'pg-master-replica-01']
pg-master-replica-standy-01 pg-master-replica-01
cn-fe1a.egron.tnx cn-e8a8.egron.tnx
2 3
